# Semantic Search

Semantic search retrieves information by understanding the meaning and context of queries using techniques like vector embeddings, enabling more accurate and relevant results beyond simple keyword matching.

## 1. Embeddings

### 1.1 What are Embeddings?

Embeddings are N-dimensional vector representations derived from text data. They capture the semantic meaning of the text, enabling similarity comparisons using distance metrics. These representations allow for efficient search and retrieval of relevant information based on textual relationships.
 

![](../obsidian/Excalidraw/Embeddings.excalidraw.svg)


### 1.2 Creating Embeddings with Ollama  

Just like chat models, embedding models can be easily downloaded and hosted on our own hardware.  

The following code snippet demonstrates how to generate embeddings using an Ollama embedding model.


In [1]:
from ollama import Client

# Download the embedding model
MODEL = "all-minilm:33m"
client = Client(host="http://localhost:11434")
client.pull(MODEL)

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [2]:
# Generate embeddings for a given text
result = client.embed(
    model=MODEL,
    input="Hello world",
)

print(f"{result.embeddings[0]}")

[-0.07589595, -0.0053512193, 0.011461193, -0.06798032, -0.0029048766, -0.18354985, 0.06604502, 0.029499538, -0.053219087, 0.08225615, -0.006884098, -0.04101772, 0.04589437, -0.028767489, -0.055228975, -0.023638006, 0.072114475, -0.027285244, -0.082989044, -0.079311885, -0.047613, 0.015250062, -0.10947498, -0.012752788, -0.03368992, -0.045844007, 0.023336934, 0.022760957, 0.07219826, -0.05898353, 0.013268217, 0.017745204, 0.12380466, 0.0014222985, -0.051148485, 0.0045865052, 0.038739294, -0.03258785, -0.011568591, 0.08333937, 0.019869013, 0.014589826, -0.033811476, 0.040321212, 0.029514974, -0.07878763, -0.006946654, -0.0368799, 0.035427053, -0.08134353, -0.0040645804, -0.052613374, 0.035524726, -0.0077684303, 0.07019797, -0.016511122, -0.010550486, 0.04372375, 0.01108922, -0.011595475, -0.025362503, 0.0069831894, 0.026311012, -0.080296054, 0.035118334, 0.017600162, 0.019714473, 0.043280464, -2.8306602e-05, -0.09367106, -0.029300535, 0.021673458, 0.059610147, -0.07154732, 0.032832302, 0

### 1.3 Calculating Similarities

To determine how similar two sentences are, we compare the distance between their embeddings. A common approach is to use **cosine similarity**, which measures the cosine of the angle between two vectors:


$\text{Cosine Similarity} = \cos(\theta) = \frac{\mathbf{a} \cdot \mathbf{b}}{|\mathbf{a}| |\mathbf{b}|}$


(An example implementation in Python is provided below.)

Visualizing the **similarity matrix** can offer deeper insights into the relationships between sentences. This allows us to identify the most semantically relevant sentences based on their contextual meaning.

In [3]:
import numpy as np


# Example function to calculate cosine similarity between multiple embeddings
def cosine_similarity(embeddings: np.ndarray) -> np.ndarray:
    """
    Calculate the cosine similarity between all pairs of embedding vectors in a 2D array.

    Parameters:
        embeddings (numpy.ndarray): 2D array where each row is an embedding.

    Returns:
        numpy.ndarray: 2D array of cosine similarities. The element at [i, j] is the cosine similarity between the ith and jth embeddings.
    """
    # Calculate the Gram matrix (dot product of each pair of embeddings)
    gram = np.dot(embeddings, embeddings.T)

    # Calculate the norms of each embedding
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)

    # Avoid division by zero by ensuring norms are at least a small epsilon
    epsilon = np.finfo(float).eps
    norms += epsilon

    # Compute the cosine similarity
    similarity = gram / (norms * norms.T)

    return similarity

### 1.3.1 Visualizing Sentence Similarities  
**a) Heatmaps**  

One effective way to analyze sentence similarities is through **heatmaps**, which provide a clear visual representation of the similarity matrix.

In [4]:
from utils import bulk_embed, plot_similarity_heatmap


example_sentences = [
    "The cat sat on the windowsill, watching the birds outside.",
    "A feline perched by the window, observing the chirping sparrows.",
    "The dog barked loudly when the mailman arrived.",
    "A postal worker delivered letters while a nearby canine growled.",
    "The sun sets in the evening, painting the sky orange and red.",
    "At dusk, the horizon glows with vibrant shades of crimson and gold.",
    "She enjoys reading mystery novels late at night.",
    "At night, she immerses herself in thrilling detective stories.",
    "The train arrived at the station five minutes late.",
    "Passengers waited patiently as the delayed locomotive approached.",
]

embeddings = bulk_embed(MODEL, example_sentences, client)


similarities = cosine_similarity(embeddings)

plot_similarity_heatmap(similarities, texts=example_sentences, limit=20)

**b) 3D-Plots**

Another powerful method for visualizing sentence similarities is **3D plots**, which offer a more dynamic perspective on the relationships between sentence embeddings.

In [5]:
from utils import plot_embeddings

plot_embeddings(embeddings=embeddings, texts=example_sentences)

### 1.4 Your Task: Multilingual Similarity Analysis  

Your task is to **visualize the similarities between sentences with similar semantic content but in different languages**. This will help assess whether our model effectively captures multilingual semantics.  

Once you have completed the analysis, repeat the process using the `granite-embedding:278m` model to compare its performance in handling multilingual embeddings.

In [6]:
### Your code goes here
sentences = [
    # English
    "The cat sat on the windowsill, watching the birds outside.",
    "A feline perched by the window, observing the chirping sparrows.",
    "The dog barked loudly when the mailman arrived.",
    "A postal worker delivered letters while a nearby canine growled.",
    # German
    "Die Katze saß auf der Fensterbank und beobachtete die Vögel draußen.",
    "Eine Katze hockte am Fenster und schaute den zwitschernden Spatzen zu.",
    "Der Hund bellte laut, als der Postbote ankam.",
    "Ein Postbote brachte Briefe, während ein Hund in der Nähe knurrte.",
    # Japanese
    "猫は窓辺に座り、外の鳥を眺めていた。",
    "窓辺にいる猫がさえずるスズメを観察していた。",
    "犬は郵便配達員が来たとき、大きな声で吠えた。",
    "郵便配達員が手紙を届けると、近くの犬が唸った。",
]

embeddings = bulk_embed("all-minilm:33m", sentences, client)

similarities = cosine_similarity(embeddings)

plot_similarity_heatmap(similarities, texts=sentences, limit=10)

embeddings = bulk_embed("granite-embedding:278m", sentences, client)

similarities = cosine_similarity(embeddings)

plot_similarity_heatmap(similarities, texts=sentences, limit=10)

### 2. Vector Stores and Search  

A **vector store** is a specialized database for managing and searching high-dimensional embeddings. Unlike traditional keyword-based search, vector stores enable **semantic search**, retrieving relevant information based on meaning rather than exact matches. This is achieved through **approximate nearest neighbor (ANN) search**, which efficiently finds semantically similar entries.  

To integrate vector stores into our system, we will again use **LangChain**, which provides seamless tools for storing and retrieving embeddings for effective semantic search.



In [7]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document


def create_vector_store(model: str = "granite-embedding:278m"):
    # Wrap our Ollama model
    embedding_provider = OllamaEmbeddings(model=model)

    # Initialize the vector store
    index = faiss.IndexFlatL2(len(embedding_provider.embed_query("hello world")))

    # Store the documents in memory for now
    vector_store = FAISS(
        embedding_function=embedding_provider,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    return vector_store


vector_store = create_vector_store()
vector_store.distance_strategy

<DistanceStrategy.EUCLIDEAN_DISTANCE: 'EUCLIDEAN_DISTANCE'>

### 2.1 Using the Vector Store  

To store text in the vector database, we wrap it in a `Document` object, which includes `page_content` for the text itself and optional `metadata` for additional context.

In [8]:
documents = []
for i, sentence in enumerate(example_sentences):
    document = Document(page_content=sentence, metadata={"document_id": i})
    documents.append(document)

ids = vector_store.add_documents(documents=documents)

### 2.2 Querying the Vector Store  

To retrieve similar documents, we can use the `similarity_search_with_score` function, which finds and ranks documents based on their relevance to a given query.

In [9]:
results = vector_store.similarity_search_with_score(query="cat", k=3)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=1.021127] A feline perched by the window, observing the chirping sparrows. [{'document_id': 1}]
* [SIM=1.030297] A postal worker delivered letters while a nearby canine growled. [{'document_id': 3}]
* [SIM=1.054524] The cat sat on the windowsill, watching the birds outside. [{'document_id': 0}]


## 3. Indexing a Codebase  

<img src="https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png" alt="Langchain Pipeline" style="width:800px;">  

Now, let's work with a real codebase and index it. As an example, we'll use the [TEI-Client](https://github.com/LLukas22/tei-client) repository, which is small and easy to understand.  

The code below clones the repository into the [`repo`](./repo/) folder. If you don’t have `git` installed, you can manually download the code from [GitHub](https://github.com/LLukas22/tei-client) and place it in the [`repo`](./repo/) directory.


In [10]:
from git import Repo
from pathlib import Path

repo_path = Path("repo").resolve()
if not repo_path.exists():
    repo_path.mkdir(parents=True)

repo_url = "https://github.com/LLukas22/tei-client.git"

try:
    Repo.clone_from(repo_url, "./repo")
except Exception as e:
    print(f"Failed to clone repository: {e}")

Failed to clone repository: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/LLukas22/tei-client.git ./repo
  stderr: 'fatal: destination path './repo' already exists and is not an empty directory.
'


### 3.1 Loading Code Files  

Langchain provides utility functions to efficiently locate and load files as `Document` objects.  

The following code demonstrates how to accomplish this:

In [11]:
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain_community.document_loaders.text import TextLoader

loader = DirectoryLoader(repo_path, glob="**/*.py", loader_cls=TextLoader)
docs = loader.load()

for doc in docs:
    print("_" * 8)
    print(f"Source: {doc.metadata["source"]}")
    print(f"Characters: {len(doc.page_content)}")

________
Source: D:\GIT\praxistage\notebooks\repo\CI\wait_for_server.py
Characters: 683
________
Source: D:\GIT\praxistage\notebooks\repo\tests\test_grpc.py
Characters: 4672
________
Source: D:\GIT\praxistage\notebooks\repo\tests\test_http.py
Characters: 4863
________
Source: D:\GIT\praxistage\notebooks\repo\tests\__init__.py
Characters: 209
________
Source: D:\GIT\praxistage\notebooks\repo\src\tei_client\models.py
Characters: 2670
________
Source: D:\GIT\praxistage\notebooks\repo\src\tei_client\__about__.py
Characters: 18
________
Source: D:\GIT\praxistage\notebooks\repo\src\tei_client\__init__.py
Characters: 479
________
Source: D:\GIT\praxistage\notebooks\repo\src\tei_client\clients\base.py
Characters: 4709
________
Source: D:\GIT\praxistage\notebooks\repo\src\tei_client\clients\grpc_client.py
Characters: 13660
________
Source: D:\GIT\praxistage\notebooks\repo\src\tei_client\clients\http_client.py
Characters: 9199
________
Source: D:\GIT\praxistage\notebooks\repo\src\tei_client\stub

### 3.2 Challenges in Embedding Code  

When adding code files to the vector store, we may encounter errors due to their large size. This happens because our embedding model has a token limit and cannot process documents exceeding that limit in a single pass.  

To resolve this, we need to split our `Document` objects into smaller, manageable chunks before embedding them. This ensures that each chunk stays within the model's token constraints while preserving the overall structure and meaning of the code.

In [12]:
from ollama import ResponseError

# This will fail if the document exceeds the maximum context length
vector_store = create_vector_store()
try:
    ids = vector_store.add_documents(docs)
except ResponseError as e:
    if e.status_code == 500:
        print("Document exceeded the maximum context length")
    else:
        print(f"Failed to add document: {e}")

Document exceeded the maximum context length


### 3.3 Your Task: Properly Splitting and Indexing Code Files  

To effectively store and search code files, we need to:  

1. **Load the code files** using Langchain’s [`GenericLoader` and `LanguageParser`](https://python.langchain.com/docs/integrations/document_loaders/source_code/). These tools help extract structured information from source code files, making them easier to process.  

2. **Split the files into manageable chunks** using [`RecursiveCharacterTextSplitter`](https://python.langchain.com/docs/integrations/document_loaders/source_code/#splitting). This ensures that each piece remains within the embedding model's token limit while maintaining logical code segments for meaningful retrieval.  

By following this approach, we can efficiently index the codebase while preserving its readability and searchability.







In [13]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

# Your code goes here
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*.py",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=1000),
)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=512, chunk_overlap=0
)

docs = splitter.split_documents(docs)
print(f"Split into {len(docs)} documents!")

Split into 245 documents!


Now we should be able to add the documents to our vector store without problems.

In [14]:
vector_store = create_vector_store()
try:
    ids = vector_store.add_documents(docs)
    print(f"Added {len(ids)} documents!")
except ResponseError as e:
    if e.status_code == 500:
        print("Document exceeded the maximum context length")
    else:
        print(f"Failed to add document: {e}")

Added 245 documents!


### 3.4 Searching the Codebase  

With our codebase successfully indexed, we can now perform searches using the vector store. Let's test it with a few queries:  

- **"Where is the `embed` function implemented?"**  
- **"How is reranking handled?"**  
- **"How can I create a client?"**  

These queries should return the most relevant code snippets, making it easy to locate specific implementations within the codebase.

In [15]:
results = vector_store.similarity_search_with_score(
    query="Where is the `embed` function implemented?", k=5
)
for doc, score in results:
    print("-"*10)
    print(f"* [SIM={score:3f}]")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}]")

----------
* [SIM=0.516049]
Content: def embed(
		self,
		text: EmbeddingInput,
		normalize: bool = True,
		truncate: bool = False,
		truncation_direction: TruncationDirection = TruncationDirection.Right,
		prompt_name: Optional[str] = None,
	) -> list[list[float]]:
		self._ensure_model_type(ModelType.Embedding)

		if isinstance(text, str):
			text = [text]
Metadata: {'source': 'D:\\GIT\\praxistage\\notebooks\\repo\\src\\tei_client\\clients\\grpc_client.py', 'language': <Language.PYTHON: 'python'>}]
----------
* [SIM=0.516049]
Content: def embed(
		self,
		text: EmbeddingInput,
		normalize: bool = True,
		truncate: bool = False,
		truncation_direction: TruncationDirection = TruncationDirection.Right,
		prompt_name: Optional[str] = None,
	) -> list[list[float]]:
		self._ensure_model_type(ModelType.Embedding)

		if isinstance(text, str):
			text = [text]
Metadata: {'source': 'D:\\GIT\\praxistage\\notebooks\\repo\\src\\tei_client\\clients\\http_client.py', 'language': <Language.PYTHON: 'p